In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [2]:
df = pd.read_csv('age_gender.csv');

In [3]:
X = pd.DataFrame(df['pixels'].str.split(' ', 48*48-1, expand=True)).astype(float)
y = df['gender']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
X_train = X_train / 255
X_test = X_test / 255
num_pixels = X_test.shape[1]

In [6]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [7]:
def acrcy(y_predicted, y_actual):
    y_predicted = np.array(pd.DataFrame(y_predicted).idxmax(1))
    y_actual = np.array(pd.DataFrame(y_actual).idxmax(1))
    y_actl = pd.Series(y_actual, name='Actual')
    y_pred = pd.Series(y_predicted, name='Predicted')
    #create confusion matrix
    print(pd.crosstab(y_actl, y_pred))
    #print accuracy of model
    print("acccuracy: " + str(metrics.accuracy_score(y_actual, y_predicted)))

In [8]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
model = baseline_model()
model.fit(X_train, y_train, validation_split=0.25, batch_size=64, epochs=8, verbose=2)

Epoch 1/8
187/187 - 20s - loss: 1.4500 - accuracy: 0.6930 - val_loss: 0.4465 - val_accuracy: 0.7975
Epoch 2/8
187/187 - 6s - loss: 0.4769 - accuracy: 0.7738 - val_loss: 1.0129 - val_accuracy: 0.5560
Epoch 3/8
187/187 - 6s - loss: 0.4480 - accuracy: 0.7972 - val_loss: 0.7707 - val_accuracy: 0.6943
Epoch 4/8
187/187 - 6s - loss: 0.4209 - accuracy: 0.8097 - val_loss: 0.3728 - val_accuracy: 0.8343
Epoch 5/8
187/187 - 6s - loss: 0.4365 - accuracy: 0.8019 - val_loss: 0.6671 - val_accuracy: 0.7079
Epoch 6/8
187/187 - 7s - loss: 0.4022 - accuracy: 0.8201 - val_loss: 0.4123 - val_accuracy: 0.8139
Epoch 7/8
187/187 - 7s - loss: 0.3921 - accuracy: 0.8218 - val_loss: 0.3790 - val_accuracy: 0.8320
Epoch 8/8
187/187 - 7s - loss: 0.4065 - accuracy: 0.8144 - val_loss: 0.4062 - val_accuracy: 0.8104


In [10]:
res = model.predict(X_test)

In [11]:
acrcy(res, y_test)

Predicted     0     1
Actual               
0          3808   295
1          1282  2438
acccuracy: 0.7984149303336316


In [12]:
X_train = X_train.values.reshape(X_train.shape[0], 48, 48, 1).astype('float32')
X_test = X_test.values.reshape(X_test.shape[0], 48, 48, 1).astype('float32')

In [13]:
def cnn_model():
    # create model
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(48, 48, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [14]:
model = cnn_model()
model.fit(X_train, y_train, validation_split=0.2, batch_size=128, epochs=12, verbose=2)

Epoch 1/12
100/100 - 23s - loss: 0.5932 - accuracy: 0.6800 - val_loss: 0.4316 - val_accuracy: 0.7970
Epoch 2/12
100/100 - 12s - loss: 0.4398 - accuracy: 0.8022 - val_loss: 0.3681 - val_accuracy: 0.8417
Epoch 3/12
100/100 - 14s - loss: 0.3945 - accuracy: 0.8215 - val_loss: 0.3590 - val_accuracy: 0.8448
Epoch 4/12
100/100 - 14s - loss: 0.3713 - accuracy: 0.8322 - val_loss: 0.3208 - val_accuracy: 0.8621
Epoch 5/12
100/100 - 14s - loss: 0.3577 - accuracy: 0.8408 - val_loss: 0.3036 - val_accuracy: 0.8618
Epoch 6/12
100/100 - 14s - loss: 0.3440 - accuracy: 0.8474 - val_loss: 0.2997 - val_accuracy: 0.8590
Epoch 7/12
100/100 - 14s - loss: 0.3333 - accuracy: 0.8544 - val_loss: 0.2932 - val_accuracy: 0.8621
Epoch 8/12
100/100 - 14s - loss: 0.3336 - accuracy: 0.8519 - val_loss: 0.2882 - val_accuracy: 0.8697
Epoch 9/12
100/100 - 14s - loss: 0.3188 - accuracy: 0.8580 - val_loss: 0.2804 - val_accuracy: 0.8722
Epoch 10/12
100/100 - 15s - loss: 0.3054 - accuracy: 0.8688 - val_loss: 0.2733 - val_accura

In [15]:
res = model.predict(X_test)

In [16]:
acrcy(res, y_test)

Predicted     0     1
Actual               
0          3620   483
1           456  3264
acccuracy: 0.8799693212322638
